## 大まかな流れ
<b>scraper</b>
- 検索ワードをもとにURL遷移
- カードのリストを取得
- カードごとのループ
    - ギャラリー無し
        - 個別の投稿ページのURLを取得
        - 別のドライバーでそのURLに飛ぶ
        - 画像URLを取得
        - Voteを取得
- 結果をcsvで保存
<br /><br />
<b>dataframe columns</b>
- url
- images
- vote

## 検索結果のページからカードの取得テスト

In [2]:
from io import StringIO, BytesIO
import os
import re
from time import sleep
import random
import chromedriver_autoinstaller
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
import datetime
import pandas as pd
import platform
from selenium.webdriver.common.keys import Keys

# import pathlib
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import urllib
from tqdm import tqdm
from bs4 import BeautifulSoup

from src import utils

In [3]:
def parse_number(text):
    if text.isdigit():
        return int(text)
    elif text[-1] == 'k':
        return int(float(text[:-1])) * 1000
    else:
        return int(float(text))
"""
prepare
"""
#make driver
driver = utils.init_driver()
image_driver = utils.init_driver()

#jump to url
url = utils.page_url('Nekomata Okayu')
driver.get(url)
print(url)

Scraping on headless mode.
Message: invalid argument: cannot parse capability: goog:chromeOptions
from invalid argument: cannot parse prefs
from invalid argument: must be a dictionary



InvalidArgumentException: Message: invalid argument: cannot parse capability: goog:chromeOptions
from invalid argument: cannot parse prefs
from invalid argument: must be a dictionary


In [4]:
for i in range(40):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)

In [5]:
#get card
cards = driver.find_elements(By.XPATH,"//div[@data-testid='post-container']")
print(len(cards))

7


In [5]:
"""
for card in cards:
"""
tweets = []
for index,card in tqdm(enumerate(cards),total=len(cards)):
    try:
        #page url
        image_base_links = card.find_elements(By.TAG_NAME,"a")
        image_base_link = ""
        for image_base in image_base_links:
            try:
                if "/r/Hololewd/comments" in image_base.get_attribute("href"):
                    image_base_link = image_base.get_attribute("href")
            except:
                pass

        if image_base_link != "":
            image_driver.get(image_base_link)

            #image link
            image_link = ""
            soup = BeautifulSoup(image_driver.page_source, 'html.parser')
            for a in soup.find_all(name="a",attrs={"rel":"noopener noreferrer"}):
                link = a.get('href')
                if '.jpg' in link or '.png' in link:
                    image_link = link

            #vote
            container = image_driver.find_element(By.XPATH,"//div[@data-testid='post-container']")
            vote_element = container.find_element(By.XPATH,".//div//div//div")
            vote = parse_number(vote_element.text)

            tweet = [
                image_base_link,
                image_link,
                vote
            ]
            tweets.append(tweet)
    except Exception as e:
        print(f'index : {index}  ',end='')
        print(e)

tweet_df = pd.DataFrame(tweets,columns=["url","images","vote"])
tweet_df


100%|██████████| 7/7 [00:29<00:00,  4.16s/it]

['https://www.reddit.com/r/Hololewd/comments/134jyjv/okayu_latex/', 'https://external-preview.redd.it/AVWbbDbJZSJ8I6aEsLCMdVtsQeof9otrra222itFtv4.png?auto=webp&v=enabled&s=6050f4e95b16b240f79dcc7032953494c5282d43', 35]
['https://www.reddit.com/r/Hololewd/comments/1348aga/okayu_relax_figure_in_latex/', 'https://external-preview.redd.it/MuNq8BzRaG222x4hfkCW1IAkbTFABjkccNQBob6Six8.jpg?auto=webp&v=enabled&s=a1a5c652d906f076e844b9ea5f320c0d07aad034', 108]
['https://www.reddit.com/r/Hololewd/comments/133uf65/okayu_on_the_glass/', 'https://i.redd.it/wlnmzyixw2xa1.png', 2000]
['https://www.reddit.com/r/Hololewd/comments/132is71/plump_and_soft_okayu/', 'https://i.redd.it/ycr6u9fphswa1.jpg', 704]
['https://www.reddit.com/r/Hololewd/comments/131p5xu/okazu/', 'https://external-preview.redd.it/8adCZpBQ3K8ibr-8BhbHBVlQYmfelr9uGagBlw9Qw4k.jpg?auto=webp&v=enabled&s=12a54aa17d3d571df2d0f559d1b3c3e77d438589', 558]
['https://www.reddit.com/r/Hololewd/comments/1308hr1/she_can_be_your_girlfriend_or_boyfrie

In [10]:
back_text = "Back to Top"
mainSoup = BeautifulSoup(driver.page_source, 'html.parser')

mainSoup.find(name="button",attrs={"tabindex":"0"})

<button class="_3KaTO_3YaHK3SMocnu8jV9 _2iuoyPiKHN3kfOoeIQalDT _3zbhtNO0bdck0oYbYRhjMC HNozj_dKjQZ59ZsfEegz8" role="button" style="margin-left: -500px;" tabindex="0">Jump to content</button>